In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report



In [7]:
df = pd.read_excel('External Validation Predictions.xlsx')

In [8]:
df.head()

,Residue_Index,UNIPROTID,Amino_Acid,Secondary_Structure,Junction_Region,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,...,Leucine_Propensity,RelSASA,Secondary_Enum,Cluster,Calculated Sum,Threshold1 >=0.1116,Regions detected for threshold1,Unnamed: 19,Threshold2 >=0.27763,Regions detected for threshold2
0,1,P|P03427|PB2,11,C,J,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.635686,0.942029,1,0.304074,1,d,NaN,1,NaN
1,2,P|P03427|PB2,4,H,J,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.689330,0.942029,0,0.167956,1,d,NaN,0,NaN
2,3,P|P03427|PB2,15,H,H,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.470975,0.130700,1,0.333016,1,d,NaN,1,NaN
3,4,P|P03427|PB2,8,H,H,0.142857,0.0,0.0,0.0,0.0,...,-0.142857,0.265970,0.130700,0,0.170216,1,d,NaN,0,NaN
4,5,P|P03427|PB2,9,H,H,0.285714,0.0,0.0,0.0,0.0,...,-0.142857,0.554636,0.130700,0,0.231534,1,d,NaN,0,NaN


In [12]:
aa_to_number = {
    'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5,
    'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10,
    'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15,
    'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20
}

number_to_aa = {v:k for k,v in aa_to_number.items()}

In [13]:
number_to_aa

{1: 'A',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'K',
 10: 'L',
 11: 'M',
 12: 'N',
 13: 'P',
 14: 'Q',
 15: 'R',
 16: 'S',
 17: 'T',
 18: 'V',
 19: 'W',
 20: 'Y'}

In [15]:
df['Amino_Acid']= df['Amino_Acid'].map(number_to_aa)

In [29]:
# Group by UNIPROTID and convert amino acids to a string (like 'MAJKB')
Amino_Acids = df.groupby('UNIPROTID')['Amino_Acid'].apply(lambda x: ''.join(map(str, x))).to_dict()

{'P|A8C8W5|M1': 'MSLLTEVETYVLSIVPSGPLKAEIAQRLEDVFAGKNTDLEALMEWLKTRPILSPLTKGILGFVFTLTVPSERGLQRRRFVQNALNGNGDPNNMDKAVKLYRKLKREITFHGAKEVALSYSAGALASCMGLIYNRMGTVTTEVAFGLVCATCEQIADSQHRSHRQMVTTTNPLIRHENRMVLASTTAKAMEQMAGSSEQAAEAMEVASQARQMVQAMRTIGTHPSSSAGLKDDLLENLQAYQKRMGVQMQRFK',
 'P|P03427|PB2': 'MERIKELRNLMSQSRTREILTKTTVDHMAIIKKYTSGRQEKNPALRMKWMMAMKYPITADKRITEMIPERNEQGQTLWSKMNDAGSDRVMVSPLAVTWWNRNGPVTSTVHYPKIYKTYFEKVERLKHGTFGPVHFRNQVKIRRRVDINPGHADLSAKEAQDVIMEVVFPNEVGARILTSESQLTTTKEKKEELQGCKISPLMVAYMLERELVRKTRFLPVAGGTSSVYIEVLHLTQGTCWEQMYTPGGEARNDDVDQSLIIAARNIVRRATVSADPLASLLEMCHSTQIGGIRMVNILRQNPTEEQAVDICKAAMGLRISSSFSFGGFTFKRTSGSSVKREEEVLTGNLQTLKIRVHEGYEEFTMVGRRATAILRKATRRLIQLIVSGRDEQSIAEAIIVAMVFSQEDCMIKAVRGDLNFVNRANQRLNPMHQLLRHFQKDAKALFQNWGIESIDNVMGMIGILPDMTPSTEMSMRGVRISKMGVDEYSSAEKIVVSIDRFLRVRDQRGNVLLSPEEVSETQGTEKLTITYSSSMMWEINGPESVLVNTYQWIIRNWETVKIQWSQNPTMLYNKMEFEPFQSLVPKAVRGQYSGFVRTLFQQMRDVLGTFDTAQIIKLLPFAAAPPKQSGMQFSSLTINVRGSGMRILVRGNSPIFNYNKTTKRLTVLGKDAGPLTEDPDEGTAGVESAVLRGFLILGKEDRRYGPALSI

In [30]:
import torch
import numpy as np
import re
import pandas as pd
from transformers import BertTokenizer, BertModel

# Load ProtBERT
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

In [31]:
def extract_and_save_embeddings(protein_name, sequence, output_file="external_training_data_protein_embeddings.csv"):
    sequence = re.sub(r"[UZOB]", "X", sequence)  # Replace rare amino acids
    sequence = " ".join(sequence)  # Ensure space-separated format

    encoded_input = tokenizer(sequence, return_tensors='pt', add_special_tokens=True)

    with torch.no_grad():
        output = model(**encoded_input)

    # Extract last hidden states
    embeddings = output.last_hidden_state.squeeze(0)  # Shape: (L, 1024)

    # Remove CLS and SEP tokens (First & Last)
    embeddings = embeddings[1:-1]  # Shape: (Sequence Length, 1024)

    # Create DataFrame with embeddings
    residues = list(sequence.replace(" ", ""))  # Remove spaces for correct residue mapping
    df = pd.DataFrame(embeddings.numpy(), columns=[f"dim_{i}" for i in range(1024)])
    df.insert(0, "Residue", residues)  # Add residue column
    df.insert(0, "Protein", protein_name)  # Add protein name column

    # Save to CSV (append if file exists)
    df.to_csv(output_file, index=False, mode='a', header=not pd.io.common.file_exists(output_file))
    print(f"Embeddings for {protein_name} saved to {output_file}")

    return embeddings

In [32]:
for name, seq in Amino_Acids.items():
    extract_and_save_embeddings(name, seq)

Embeddings for P|A8C8W5|M1 saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03427|PB2 saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03428|PB2 saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03430|RDRP saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03431|RDRP saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03433|PA saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03452|HEMA saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03454|HEMA saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03466|NCAP saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03468|NRAM saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03470|NRAM saved to external_training_data_protein_embeddings.csv
Embeddings for P|P03485|M1 saved to external_training_data_protein_embeddings.csv
